In [451]:
#Decision forest of OneAPI on Insurance Fraud Detection dataset

In [355]:
import daal4py as d4p
import numpy as np
import pandas 

In [356]:
read_csv = pandas.read_csv("Module 11\\[Dataset]_Module11_(Insurance).csv")

In [357]:
features = []
for col in read_csv.columns:
    if col != 'fraud_reported':
        features.append(col)

target = 'fraud_reported'

X = read_csv[features]
y = read_csv[target]

In [358]:
#Syntax: X_train, X_test, y_train, y_test = train_test_split()
from sklearn.model_selection import train_test_split
data, prunedata, labels, prunelabels = train_test_split(X, y, random_state = 1)

In [359]:
# Lets configure a training object (5 classes)
train_algo = d4p.decision_forest_classification_training(5,
                                                         nTrees=10,
                                                         minObservationsInLeafNode=8,
                                                         featuresPerNode=3,
                                                         engine = d4p.engines_mt19937(seed=777),
                                                         varImportance='MDI',
                                                         bootstrap=True,
                                                         resultsToCompute='computeOutOfBagError')

In [360]:
train_result = train_algo.compute(data, labels)

In [361]:
predict_algo = d4p.decision_forest_classification_prediction(nClasses=5)
predict_result = predict_algo.compute(prunedata, train_result.model)
assert(predict_result.prediction.shape == (prunedata.shape[0], 1))
#We pass x_test(prunedata here) in predict_result 

In [362]:
#And then we compare y_pred(predict_result.prediction here) with y_test(prunelabels here)
print("\nRandom forest prediction results (first 10 rows):\n", predict_result.prediction[0:10])
print("\nGround truth (first 10 rows):\n", prunelabels[0:10])


Random forest prediction results (first 10 rows):
 [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

Ground truth (first 10 rows):
 507    0
818    0
452    0
368    1
242    0
929    1
262    1
810    0
318    0
49     0
Name: fraud_reported, dtype: int64


In [363]:
from sklearn.metrics import confusion_matrix, classification_report
a = predict_result.prediction.reshape(-1)
b = prunelabels

In [364]:
confusion_matrix(b,a)

array([[178,   2],
       [ 59,  11]], dtype=int64)

In [365]:
print(classification_report(b,a))

              precision    recall  f1-score   support

           0       0.75      0.99      0.85       180
           1       0.85      0.16      0.27        70

    accuracy                           0.76       250
   macro avg       0.80      0.57      0.56       250
weighted avg       0.78      0.76      0.69       250

